In [ ]:
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras tf-nightly-gpu deeppavlov

In [1]:
import tensorflow as tf
import numpy as np
tf.__version__

'1.14.1-dev20190524'

In [2]:
tf.keras.utils.get_file(
    fname='rubert_cased_L-12_H-768_A-12_v1.tar.gz',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

'./models/rubert_cased_L-12_H-768_A-12_v1.tar.gz'

In [2]:
# some constants to construct input tensors
BATCH_SIZE = 4
MAX_LEN = 512

input_ids = np.array([[1, 3, 2, 5] * (MAX_LEN // 4),
                      [1, 2, 3] * (MAX_LEN // 4) + [0] * (MAX_LEN // 4)] * (BATCH_SIZE // 2))
input_mask = np.int32(np.not_equal(input_ids, 0))

transparent_input_mask = np.ones_like(input_ids)

sep_ids = np.int32(np.equal(input_ids, 103))

token_types = np.cumsum(sep_ids, axis=1) - sep_ids

# Refactored implementation

In [3]:
from bert_dp import bert
keras_graph = tf.compat.v1.Graph()

In [4]:
with keras_graph.as_default():
    bm_ = bert.BERT(trainable=False, name='bert')
    keras_sess = tf.compat.v1.Session()
#     inp = tf.keras.layers.Input(shape=(None,), batch_size=BATCH_SIZE, dtype=tf.int32)
    out = bm_(input_ids)  # bm_(inp)

W0524 16:22:22.119541 140702121703232 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/embeddings.py:27: The name tf.keras.initializers.TruncatedNormal is deprecated. Please use tf.compat.v1.keras.initializers.TruncatedNormal instead.

W0524 16:22:22.120918 140702121703232 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:94: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
keras_graph.get_collection('trainable_variables')

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(119547, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/bias:0' shape=(768,) dtype=float32>,
 <tf.

In [6]:
with keras_graph.as_default():
    from tensorflow.python.ops import variables
    keras_vs = variables._all_saveable_objects()
    keras_saver = tf.compat.v1.train.Saver(keras_vs)
    keras_saver_checkpoint_path = 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
    keras_saver.restore(keras_sess, keras_saver_checkpoint_path)

W0524 16:22:27.660011 140702121703232 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [8]:
with keras_graph.as_default():
    # object-based loading/saving... not currently working
#     checkpoint_path = 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
#     checkpoint = tf.train.Checkpoint(model=bm_)
#     status = checkpoint.restore(checkpoint_path).assert_consumed().initializer_or_restore(session=keras_sess)
    # conversion for future usage
    bm_.save_weights('models/rubert_converted_checkpoint/bert_model')

# Original implementation

In [7]:
from bert_dp.modeling import BertModel, BertConfig
orig_graph = tf.compat.v1.Graph()

In [8]:
with orig_graph.as_default():

    input_ids_ph = tf.placeholder(shape=(None, None), dtype=tf.int32, name='ids_ph')
    input_masks_ph = tf.placeholder(shape=(None, None), dtype=tf.int32, name='masks_ph')
    token_types_ph = tf.placeholder(shape=(None, None), dtype=tf.int32, name='token_types_ph')

    orig_sess = tf.compat.v1.Session()
    bm = BertModel(config=BertConfig.from_json_file('models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json'),
                   is_training=False,
                   input_ids=input_ids_ph,
                   input_mask=input_masks_ph,
                   token_type_ids=token_types_ph
                   )

W0524 16:22:37.751827 140702121703232 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/modeling.py:92: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0524 16:22:37.756990 140702121703232 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/modeling.py:178: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0524 16:22:37.758948 140702121703232 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/modeling.py:418: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0524 16:22:37.787749 140702121703232 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/modeling.py:499: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

W0524 16:22:38.185981 140702121703232 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFl

In [9]:
orig_graph.get_collection('trainable_variables')

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(119547, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/bi

In [10]:
with orig_graph.as_default():
    from tensorflow.python.ops import variables
    orig_vs = variables._all_saveable_objects()
    orig_saver = tf.compat.v1.train.Saver(orig_vs)
    orig_saver_checkpoint_path = 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
    orig_saver.restore(orig_sess, orig_saver_checkpoint_path)

# Checkpoint exploration

In [13]:
checkpoint_path = 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
var_names = tf.train.list_variables(checkpoint_path)
checkpoint_reader = tf.train.load_checkpoint(checkpoint_path)

In [14]:
checkpoint_vars = set()
for v, _ in var_names:
    checkpoint_vars.add(v)

In [15]:
checkpoint_vars

{'bert/embeddings/LayerNorm/beta',
 'bert/embeddings/LayerNorm/gamma',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/position_embeddings/AdamWeightDecayOptimizer',
 'bert/embeddings/position_embeddings/AdamWeightDecayOptimizer_1',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/word_embeddings',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/output/LayerNorm/beta',
 

In [16]:
len(checkpoint_vars)

209

In [17]:
bert_model_vars = set()
for v in bm_.variables:
    bert_model_vars.add(v.name.split(':')[0])

In [18]:
bert_model_vars

{'bert/embeddings/LayerNorm/beta',
 'bert/embeddings/LayerNorm/gamma',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/word_embeddings',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/output/LayerNorm/beta',
 'bert/encoder/layer_0/output/LayerNorm/gamma',
 'bert/encoder/layer_0/output/dense/bias',
 'bert/encoder/layer_0/output/dense/kernel

In [19]:
len(bert_model_vars)

199

# Equality testing

In [11]:
for i in range(len(orig_graph.get_collection('trainable_variables'))):
    with orig_graph.as_default():
        orig_v = orig_sess.run(orig_graph.get_collection('trainable_variables')[i].read_value())
    with keras_graph.as_default():
        keras_v = keras_sess.run(keras_graph.get_collection('trainable_variables')[i].read_value())
    print(keras_graph.get_collection('trainable_variables')[i])
    try:
        np.testing.assert_allclose(keras_v, orig_v)
    except:
        print("LayerNorm variables orders are different")

<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(119547, 768) dtype=float32>
<tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32>
<tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32>
<tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_0/at

<tf.Variable 'bert/encoder/layer_4/output/dense/kernel:0' shape=(3072, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_4/output/dense/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_4/output/LayerNorm/gamma:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/encoder/layer_4/output/LayerNorm/beta:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/encoder/layer_5/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_5/attention/self/query/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_5/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_5/attention/self/key/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_5/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_5/attention/self/value/bias:0' shape=(768,) dtype=float32>


<tf.Variable 'bert/encoder/layer_9/output/dense/kernel:0' shape=(3072, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_9/output/dense/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_9/output/LayerNorm/gamma:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/encoder/layer_9/output/LayerNorm/beta:0' shape=(768,) dtype=float32>
LayerNorm variables orders are different
<tf.Variable 'bert/encoder/layer_10/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_10/attention/self/query/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_10/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_10/attention/self/key/bias:0' shape=(768,) dtype=float32>
<tf.Variable 'bert/encoder/layer_10/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>
<tf.Variable 'bert/encoder/layer_10/attention/self/value/bias:0' shape=(768,) dtype=flo

In [13]:
with orig_graph.as_default():
    current_mask = transparent_input_mask
    emb_output = orig_sess.run(bm.get_embedding_output(), feed_dict={input_ids_ph: input_ids,
                                                                     input_masks_ph: current_mask,
                                                                     token_types_ph: token_types})
    emb_output_second = orig_sess.run(bm.get_embedding_output(), feed_dict={input_ids_ph: input_ids,
                                                                            input_masks_ph: current_mask,
                                                                            token_types_ph: token_types})
    all_encoder_layers = orig_sess.run(bm.get_all_encoder_layers(), feed_dict={input_ids_ph: input_ids,
                                                                               input_masks_ph: current_mask,
                                                                               token_types_ph: token_types})
    pooled_output = orig_sess.run(bm.get_pooled_output(), feed_dict={input_ids_ph: input_ids,
                                                                     input_masks_ph: current_mask,
                                                                     token_types_ph: token_types})

In [14]:
with keras_graph.as_default():
    first_transformer_block = bm_.layers[3].layers[0]
    emb_output_ = keras_sess.run(
        bm_.embedding_dropout(
            bm_.embedding_layer_norm(
                bm_.emb(input_ids, mask=current_mask)
            ), training=False
        )
    )
    emb_output_second_ = keras_sess.run(
        bm_.embedding_dropout(
            bm_.embedding_layer_norm(
                bm_.emb(input_ids, mask=current_mask)
            ), training=False
        )
    )
    first_layer_output_ = keras_sess.run(first_transformer_block(emb_output_))

    pooled_output_ = keras_sess.run(bm_(input_ids, mask=current_mask))

In [15]:
np.testing.assert_allclose(emb_output_second, emb_output)

In [16]:
np.testing.assert_allclose(emb_output_second_, emb_output_)

In [19]:
np.testing.assert_allclose(emb_output_, emb_output, atol=1e-6)  # TODO: investigate differences

In [21]:
np.testing.assert_allclose(first_layer_output_, all_encoder_layers[0], atol=1e-5)

In [22]:
np.testing.assert_allclose(pooled_output_, pooled_output, atol=1e-6)

### The end!

# Debugging stuff

In [31]:
from bert_dp.modeling import attention_layer, layer_norm, create_attention_mask_from_input_mask

In [34]:
from bert_dp.activations import gelu

In [32]:
g = tf.compat.v1.Graph()

In [33]:
with g.as_default():
    sess = tf.compat.v1.Session()
    emb = tf.convert_to_tensor(emb_output)
    mask_tensor = tf.convert_to_tensor(current_mask)

In [35]:
with g.as_default():
    reuse = tf.compat.v1.AUTO_REUSE
    with tf.compat.v1.variable_scope("bert/encoder/layer_0/attention", reuse=reuse):
        with tf.compat.v1.variable_scope("self", reuse=None):
            attention_head = attention_layer(
                from_tensor=emb,
                to_tensor=emb,
                attention_mask=create_attention_mask_from_input_mask(from_tensor=emb, to_mask=mask_tensor),
                num_attention_heads=12,
                size_per_head=(768 // 12),
                attention_probs_dropout_prob=0.0, # !
    #             initializer_range=initializer_range,
                do_return_2d_tensor=False,
                batch_size=BATCH_SIZE,
                from_seq_length=MAX_LEN,
                to_seq_length=MAX_LEN
            )
        with tf.compat.v1.variable_scope("output", reuse=None):
            attention_output = tf.layers.dense(
                      attention_head,
                      768,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

#             attention_output = tf.layers.dropout(attention_output, 0.1)
            attention_output = layer_norm(attention_output + emb)

    # The activation is only applied to the "intermediate" hidden layer.
    with tf.variable_scope("bert/encoder/layer_0/intermediate", reuse=reuse):
        intermediate_output = tf.layers.dense(attention_output,
                                              3072,
                                              activation=gelu,
                                              kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

    # Down-project back to `hidden_size` then add the residual.
    with tf.variable_scope("bert/encoder/layer_0/output", reuse=reuse):
        layer_output = tf.layers.dense(
            intermediate_output,
            768,
            kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))
#         layer_output = tf.layers.dropout(layer_output, 0.1)
        layer_output = layer_norm(layer_output + attention_output)

In [36]:
g.get_collection('trainable_variables')

[<tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/output/dense/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/output/dense/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/output/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/output/LayerNorm/gamma:0' shape=(768,) dty

In [37]:
with g.as_default():
    saver = tf.compat.v1.train.Saver(variables._all_saveable_objects())
    saver_checkpoint_path = 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
    saver.restore(sess, saver_checkpoint_path)

In [38]:
with g.as_default():
    mhsa = sess.run(attention_head)
    attention_output_res = sess.run(attention_output)
    intermediate_output_res = sess.run(intermediate_output)
    layer_output_res = sess.run(layer_output)
with keras_graph.as_default():
    mhsa_ = keras_sess.run(first_transformer_block.mhsa(emb_output, mask=current_mask))

    attention_output_ = first_transformer_block.layernorm1(
        emb_output + first_transformer_block.dropout1(
            first_transformer_block.dense(mhsa), training=False
        )
    )
    
    attention_output_res_ = keras_sess.run(attention_output_)

    intermediate_output_res_ = keras_sess.run(first_transformer_block.pff1(attention_output_res_))
    
    layer_output_ = first_transformer_block.layernorm2(
        attention_output_res_ + first_transformer_block.dropout2(
            first_transformer_block.pff2(intermediate_output_res_), training=False
        )
    )
    layer_output_res_ = keras_sess.run(layer_output_)

In [39]:
np.testing.assert_allclose(mhsa_, mhsa)

In [40]:
np.testing.assert_allclose(attention_output_res_, attention_output_res)

In [41]:
np.testing.assert_allclose(intermediate_output_res_, intermediate_output_res)

In [43]:
np.testing.assert_allclose(layer_output_res_, layer_output_res)